# Extração da Dados
- Via Spotify API

In [1]:
import json

with open('config.json') as json_file:
    config = json.load(json_file)
    


In [2]:
import requests

def request_data(url,params=None)-> dict:
    headers = {
        'Accept': 'application/json',
        'Content-Type': 'application/json',
        'Authorization': config['token'],
    }
    
    response = requests.get(url=url, headers=headers, params=params)

    return response.json()

## Dados de Usuário

In [4]:
data = request_data(url='https://api.spotify.com/v1/me')

user = {
    'images' :data['images'][0]['url'],
    'display_name' :data['display_name'],
    'followers' :data['followers']['total']
    }


In [6]:
with open("user.json","w") as out:
    json.dump(user, out)

## Meus Álbuns Salvos

In [7]:
data = request_data(url = 'https://api.spotify.com/v1/me/albums',
                    params=(('limit', '50'),))

In [8]:
def get_into_album(item)->dict:
    d = {
        "id" : item['id'],
        "external_urls" : item['external_urls']['spotify'],
        "images" : item['images'][0]['url'],
        "name" : item['name'],
        "artist" : item['artists'][0]['name'],
        "id_artist" : item['artists'][0]['id']
    }
    return d

albuns = []

for item in data['items']:
    d = get_into_album(item['album'])
    albuns.append(d)


In [9]:
with open("albuns.json","w") as out:
    json.dump(albuns, out)

## Artistas de Álbuns


In [10]:
import numpy as np

def get_artist_album(id:str)->dict:
    data = request_data(url = f'https://api.spotify.com/v1/artists/{id}')

    d = {
        'genres':data['genres'],
        'external_urls':data['external_urls']['spotify'],
        'id':data['id'],
        'images':data['images'][0]['url'],
        'name':data['name']
    }
    return d
    

ids_atists = np.unique([item['id_artist'] for item in albuns])

artists_albuns = [get_artist_album(id) for id in ids_atists]

In [11]:
with open("artists_albuns.json","w") as out:
    json.dump(artists_albuns, out)

# Minhas Playslists

In [12]:
data = request_data(url = 'https://api.spotify.com/v1/me/playlists',
                    params=(('limit', '50'),))

In [13]:
def get_info_playlist(item:str)->dict:
    d = {
        'external_urls':item['external_urls']['spotify'],
        'id':item['id'],
        'images':item['images'][0]['url'],
        'name':item['name']
    }
    return d 

playlist = []

for item in data['items']:
    d = get_info_playlist(item)
    playlist.append(d)

In [14]:
with open("playlist.json","w") as out:
    json.dump(playlist, out)

## Músicas de Playlists

In [15]:
def get_music_playlist(id:str)->list:
    data = request_data(url = f'https://api.spotify.com/v1/playlists/{id}/tracks',
                        params=(('limit', '50'),('offset', '0'),))
    
    items = data['items']
    return items
    
def refactor_track(track:dict,id_playlist:str)->dict:
    d = {
        'id_playlist':id_playlist,
        'id_album':track['track']['album']['id'],
        'name_album':track['track']['album']['name'],
        'external_urls_album': track['track']['album']['external_urls']['spotify'],
        'external_urls_artists':track['track']['artists'][0]['external_urls']['spotify'],
        'id_artists':track['track']['artists'][0]['id'],
        'name_artists':track['track']['artists'][0]['name'],
        'track_duration':track['track']['duration_ms'],
        'explicit':track['track']['explicit'],
        'external_urls_track': track['track']['external_urls']['spotify'],
        'id_track':track['track']['id'],
        'name_track':track['track']['name']
        }
    return d


all_tracks = []

for item in playlist:
    id = item['id']
    tracks = get_music_playlist(id)
    for track in tracks:
        d = refactor_track(track,id)
        all_tracks.append(d)




In [16]:
with open("all_tracks.json","w") as out:
    json.dump(all_tracks, out)

## Artistas de Playlists

In [17]:
import pandas as pd 

def get_artist_playlist(id_artist:str, id_playlist:str)->dict:
    data = request_data(url = f'https://api.spotify.com/v1/artists/{id_artist}')

    d = {
        'id_playlist':id_playlist,
        'genres':data['genres'],
        'external_urls':data['external_urls']['spotify'],
        'id':data['id'],
        'images':data['images'],
        'name':data['name']
    }
    return d
    

params = [[item['id_artists'],item['id_playlist']] for item in all_tracks]

info = pd.DataFrame(data=params,columns=['id_artists','id_playlist']).drop_duplicates()


artists_playlists = []

for index,row in info.iterrows():
    d = get_artist_playlist(row['id_artists'], row['id_playlist'])
    artists_playlists.append(d)


In [18]:
with open("artistis_playlists.json","w") as out:
    json.dump(artists_playlists, out)